# Чистый ноутбук для тренировки катбуста
Оставил только тренировку и вывод метричек

In [8]:
from json import load as json_load

from catboost import CatBoostClassifier, Pool
from numpy import zeros
from pandas import DataFrame, read_csv
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from tabulate import tabulate
from transliterate import translit

In [2]:
%%time
df_train = read_csv("../data/train_preprocessed.csv")
df_test = read_csv("../data/val_preprocessed.csv")

# df_train['description'] = df_train['description'].replace(r'[\W_]+', ' ', regex = True).str.lower()
# df_val['description'] = df_val['description'].replace(r'[\W_]+', ' ', regex = True).str.lower()
y_test = df_test.is_bad
y_train = df_train.is_bad

df_train = df_train.loc[:, df_train.columns != 'is_bad']
df_test = df_test.loc[:, df_test.columns != 'is_bad']

CPU times: user 5.73 s, sys: 316 ms, total: 6.04 s
Wall time: 6.05 s


In [30]:
%%timet
with open('../lib/models/regexps/regexp.json') as json_file:
    regexps = json_load(json_file)
    json_file.close()

CPU times: user 618 µs, sys: 0 ns, total: 618 µs
Wall time: 315 µs


In [31]:
regexp_names = regexps.keys()

Тут я не удаляю категории сразу, чтобы потом вывести метрички по ним

In [32]:
train_pool_text = Pool(data=df_train.drop(['category'], axis=1), label=y_train, text_features=['title_and_description'])
test_pool_text = Pool(data=df_test.drop(['category'], axis=1), label=y_test, text_features=['title_and_description'])

In [3]:
catboost = CatBoostClassifier().load_model('../lib/models/catboost_classifier.cbm')

In [46]:
%%time
catboost = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    random_strength=0.01,
    depth=8,
    l2_leaf_reg=15,
    eval_metric='AUC',
    verbose=0,
    task_type='CPU',
    thread_count=-1,
)
catboost.fit(train_pool_text)

CPU times: user 1h 6min 38s, sys: 13.5 s, total: 1h 6min 52s
Wall time: 4min 42s


In [47]:
def metrics_printer(X, y, model):
    overall_table = []
    cats_table = []
    pred_probas = zeros(y.shape)
    labels_pred = zeros(y.shape)
    for category in X["category"].unique():
        cat_name = translit(category.lower().replace(" ", "_"), 'ru', reversed=True)
        pred_probas[X["category"] == category] = model.predict_proba(X.drop(['category'], axis=1)[X["category"] == category])[:, 1]
        labels_pred[X["category"] == category] = model.predict(X.drop(['category'], axis=1)[X["category"] == category])
        rocauc_category = roc_auc_score(y[X["category"] == category], pred_probas[X["category"] == category])
        f1_category = f1_score(y[X["category"] == category], labels_pred[X["category"] == category])
        accuracy_category = accuracy_score(y[X["category"] == category], labels_pred[X["category"] == category])
        cats_table.append([cat_name, rocauc_category, f1_category, accuracy_category])
    rocauc = roc_auc_score(y, pred_probas)
    f1 = f1_score(y, labels_pred)
    accuracy = accuracy_score(y, labels_pred)
    overall_table.append([rocauc, f1, accuracy])
    print("Categories table:")
    print(tabulate(cats_table, headers=["Category", "AUC", 'f1', 'accuracy'], tablefmt='orgtbl') + '\n')
    print("Overall table:")
    print(tabulate(overall_table, headers=['AUC', 'f1', 'accuracy'], tablefmt='orgtbl'))

In [48]:
%%time
metrics_printer(df_test, y_test, catboost)

Categories table:
| Category             |      AUC |       f1 |   accuracy |
|----------------------+----------+----------+------------|
| transport            | 0.982665 | 0.897846 |   0.939603 |
| dlja_biznesa         | 0.88068  | 0.551724 |   0.910035 |
| dlja_doma_i_dachi    | 0.92784  | 0.786127 |   0.922836 |
| lichnye_veschi       | 0.835472 | 0.625    |   0.915014 |
| uslugi               | 0.849788 | 0.759322 |   0.78869  |
| bytovaja_elektronika | 0.940927 | 0.546479 |   0.942253 |
| nedvizhimost'        | 0.907043 | 0.744828 |   0.789414 |
| hobbi_i_otdyh        | 0.906747 | 0.481928 |   0.892095 |
| rabota               | 0.864855 | 0.742222 |   0.786765 |
| zhivotnye            | 0.913923 | 0.833333 |   0.838095 |

Overall table:
|      AUC |       f1 |   accuracy |
|----------+----------+------------|
| 0.944708 | 0.794485 |   0.903615 |
CPU times: user 2.97 s, sys: 68 ms, total: 3.04 s
Wall time: 636 ms


In [49]:
catboost.save_model('../lib/models/catboost_classifier.cbm')

Тут мне было интересно, сколько же времени работает предсказание

In [9]:
preds = catboost.predict_proba(df_test.drop('category', axis=1))[:, 1]

In [13]:
DataFrame(zip(range(len(preds)), preds), columns=['index', 'prediction'])

,index,prediction
0,0,0.936225
1,1,0.009671
2,2,0.252154
3,3,0.859289
4,4,0.128627
...,...,...
16232,16232,0.288358
16233,16233,0.043034
16234,16234,0.017408
16235,16235,0.007958
